In [1]:
'''Predicting Mortality across Germany wih different AI Methods
'''

import numpy as np
import pandas as pd
import os
import seaborn as sns
from matplotlib import pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import impute
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.svm import SVR, NuSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
import dbf
from pygam import GAM, s, f, LinearGAM
import xgboost
import joblib
import geopandas as gpd
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from pprint import pprint
from stabilityselection import Colinearity_Remover

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# Prediction

In [40]:
def Create_grid_serach(method):
    
    ''' This function makes a grid for hyper-parameter tuning based on parameters of each method.
        
        Input:
        --------------------------
        method (str): name of the method
        
        Output:
        ---------------------------
        grid (dictionary): a grid containing ranges for hyper-parameters w.r.t. the specified method
        
    '''
    
    # Create a parameter grid
       
    if (method == 'LR_Lasso'):
        # n_alpha: Number of alphas along the regularization path
        # eps: Length of the path
        # cv: number of folds in built-in cross-validation
        grid = {'n_alphas': [100, 200, 300],
               'eps': [0.01, 0.001],
               'cv': [5, 10]}
        
    if (method == 'LR_Ridge'):
        # alpha: regularization strength
        grid = {'cv': [5, 10]}
        
    if (method == 'LR_Elastic'):
        # l1_ratio: scaling between Ridge and Lasso
        # n_alpha: Number of alphas along the regularization path
        # eps: Length of the path
        grid = {'l1_ratio': [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
        'n_alphas': [100, 200, 300],
        'eps': [0.01, 0.001],
        'cv': [5, 10]
        }
        
    if (method == 'RF'):
        # Number of trees in random forest
        n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
        # Number of features to consider at every split
        max_features = ['auto', 'sqrt']
        # Maximum number of levels in tree
        max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
        max_depth.append(None)
        # Minimum number of samples required to split a node
        min_samples_split = [2, 5, 10]
        # Minimum number of samples required at each leaf node
        min_samples_leaf = [1, 2, 4]
        # Method of selecting samples for training each tree
        bootstrap = [True, False]
        # Create the random grid
        grid = {'n_estimators': n_estimators,
                       'max_features': max_features,
                       'max_depth': max_depth,
                       'min_samples_split': min_samples_split,
                       'min_samples_leaf': min_samples_leaf,
                       'bootstrap': bootstrap}
        
    elif (method == 'AdaB'):
        # Number of trees in AdaBoost
        n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
        # The loss function to use when updating the weights after each boosting iteration
        loss = ['linear', 'square', 'exponential']
        # Weight applied to each regressor at each boosting iteration
        learning_rate = [float(x) for x in np.linspace(0.1, 2, num = 20)]
        # Create the random grid
        grid = {'learning_rate': learning_rate,
                       'loss': loss,
                       'n_estimators': n_estimators}
        
    elif (method == 'XGBoost'):
        # Number of gradient boosted trees
        n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]
        # Boosting learning rate
        learning_rate = [float(x) for x in np.linspace(0.01, 0.2, num = 20)]
        # Create the random grid
        grid = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': learning_rate,
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': n_estimators}
        
    elif (method == 'SVR'):
        # C parameter
        C = [0.1, 1, 10, 100]
        # kernel
        kernel = ['linear', 'poly', 'rbf', 'sigmoid']
        # Create the random grid
        grid = { 
            'C': C,
           'kernel': kernel
                      }
        
    elif (method == 'KNN'):
        # Algorithm used to compute the nearest neighbors
        algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
        n_beighbors = list(range(1,30))
#         Power parameter for the Minkowski metric. When p = 1, this is equivalent to using manhattan_distance (l1),
#         and euclidean_distance (l2) for p = 2. For arbitrary p, minkowski_distance (l_p) is used.
        p = [1, 2]
#         Weight function used in prediction
        weight = ['uniform', 'distance']
#         Leaf size passed to BallTree or KDTree. 
        leaf_size = list(range(1,50))
        grid = {
            'algorithm': algorithm,
            'leaf_size': leaf_size,
                       'n_neighbors': n_beighbors,
                       'p': p,
                       'weights': weight
                      }
    elif (method == 'MLP'):
        
        grid = {
            'hidden_layer_sizes': [(150,100,50), (120,80,40), (100,50,30), (100,)],
            'max_iter': [50, 100],
            'activation': ['identity', 'logistic', 'tanh', 'relu'],
            'solver': ['sgd', 'adam', 'invscaling'],
            'alpha': [0.0001, 0.05],
            'learning_rate': ['constant','adaptive'],
            }
    
    pprint(grid)
    
    return grid

In [25]:
def RandomSearch_Param_Tuning(model, method, itr, X, y, grid = None, n_cv = 5, verbose = 2, rand_state = 42, jobs = -1):
    
    ''' This function performs a random search for hyper-parameter tuning for the specified method.
        
        Input:
        --------------------------
        model (e.g. SKlearn models): it is a trained model
        
        method (str): name of the method
        
        itr (int): number of iterations
        
        X (pandas dataframe): data for the predictors
        
        y (pandas dataframe, pandas serie): output
        
        grid (dictionary): a grid containing ranges for hyper-parameters w.r.t. the specified method (optional)
        
        n_cv (int): number of folds in cross-validation
        
        verbose (int): Controls the verbosity, the higher, the more messages.
        
        rand_state (int): an integer to set and make the result reproducable
                
        jobs (int): number of cpus to specify for the parallelization 
        
        Output:
        ---------------------------
        random_model.best_estimator_ (sklearn model): the best fitted model employing random search
        
        random_model.best_params_ (dictionary): list of the parameters for the best fitted model employing random search
        
        random_model (list of sklearn models): list of all tried methods employing random search
        
    '''
    
    print('Runnig Random Search for hyper-parameter Tuning when Scoring is R-squared for ' + method + ' ..... ')
    
    if not grid:
        grid = Create_grid_serach(method)

    random_model = RandomizedSearchCV(estimator = model, param_distributions = grid, n_iter = itr, cv = n_cv,
                                      scoring= 'r2', verbose=2, random_state=42, n_jobs = -1)
    # Fit the random search model
    random_model.fit(X, y)
    
    return random_model.best_estimator_, random_model.best_params_, random_model


In [4]:
def GridSearch_Param_Tuning(model, method, X, y, grid = None, n_cv = 5, verbose = 2, jobs = -1):

    ''' This function performs a grid search for hyper-parameter tuning for the specified method.
        
        Input:
        --------------------------
        model (e.g. SKlearn models): it is a trained model
        
        method (str): name of the method
        
        X (pandas dataframe): data for the predictors
        
        y (pandas dataframe, pandas serie): output
        
        grid (dictionary): a grid containing ranges for hyper-parameters w.r.t. the specified method (optional)
        
        n_cv (int): number of folds in cross-validation
        
        verbose (int): Controls the verbosity, the higher, the more messages.
        
        jobs (int): number of cpus to specify for the parallelization 
        
        Output:
        ---------------------------
        grid_search.best_estimator_ (sklearn model): the best fitted model employing grid search
        
        grid_search.best_params_ (dictionary): list of the parameters for the best fitted model employing grid search
        
        grid_search (list of sklearn models): list of all tried methods employing random search
        
    '''    
    
    print('Runnig Grid Search for Hyper-parameter Tuning  when Scoring is R-squared for ' + method + ' ..... ')
    
    if not grid:
        grid = Create_grid_serach(method)
    
    grid_search = GridSearchCV(estimator = model, param_grid = grid, cv = n_cv, scoring= 'r2', n_jobs = jobs,
                               verbose = verbose)
    grid_search.fit(X, y)
    
    return grid_search.best_estimator_, grid_search.best_params_, grid_search


In [46]:
def Build_fit_model(method, X_train, y_train):
    
    ''' This function builds and fits a model using the data provided.
        
        Input:
        --------------------------
        method (str): name of the method
        
        X_train (pandas dataframe): data for the predictors
        
        y_train (pandas dataframe, pandas serie): output data
        
        
        Output:
        ---------------------------
        model (e.g. sklearn model): it is a trained model w.r.t. the method you specify
        
    '''
        
    if method == 'LR':
        model = linear_model.LinearRegression()
    elif method == 'LR_Ridge':
        model = linear_model.RidgeCV()
    elif method == 'LR_Lasso':
        model = linear_model.LassoCV()
    elif method == 'LR_Elastic':
        model = linear_model.ElasticNetCV()
    elif method == 'GAM':
        model = GAM().fit(X_train, y_train)
    elif method == 'RF':
        model = RandomForestRegressor()
    elif method == 'AdaB':
        model = AdaBoostRegressor()
    elif method == 'XGBoost':
        model = xgboost.XGBRegressor()
    elif method == 'SVR':
        model = SVR()
    elif method == 'KNN':
        model = KNeighborsRegressor()
    elif method == 'MLP':
        model = MLPRegressor()
    
    if method == 'GAM':
        model.gridsearch(X_train, y_train)
    elif method in  ['LR']:
        model = model.fit(X_train, y_train)
    elif method in ['LR_Ridge', 'LR_Lasso', 'LR_Elastic', 'KNN', 'SVR', 'RF', 'AdaB', 'XGBoost', 'MLP']:
        model, parameters, all_models = RandomSearch_Param_Tuning(model, method, 100, X_train, y_train)
        
    print('Parameters for ' + method)
    pprint(model.get_params())
    
    os.makedirs("trained_models", exist_ok=True)
    joblib.dump(model, 'trained_models/' + method + '_trained_model.sav')
    
    return model

In [6]:
def save_model(model, file_path, model_name = None):
    
    ''' This function saves a built and fitted model as .sav using joblib.
        
        Input:
        --------------------------
        model (e.g. SKlearn models): it is a trained model
        
        file_path (str): where to save the model
        
        model_name (str): name of the model (optional)
        
        
        Output:
        ---------------------------
        None
    '''
    
    joblib.dump(model, file_path + model_name + '_trained_model.sav')

In [7]:
def load_model(file_path):
    
    ''' This function loads a saved built and fitted model as .sav using joblib.
        
        Input:
        --------------------------
       
        file_path (str): where to save the model
        
        
        Output:
        ---------------------------
        loaded_model (e.g. SKlearn models): a built and fitted model

    '''
        
    loaded_model = joblib.load(file_path)
    return loaded_model

In [10]:
def Prediction(method, X_train, X_val, y_train, y_val, X_test, y_test, output_variable, output_path, data_training,
               data_test, grid_5km_shp, model = None):
        
    ''' This function makes prediction for the provided data and reports three metrics, i.e. MAE, MSE, R-squared.
        
        Input:
        --------------------------
        
        method (str): name of the method
        
        X_train (pandas dataframe): data for the training
        
        X_val (pandas dataframe): data for the validation
        
        y_train (pandas dataframe, pandas serie): output data for the training
        
        y_val (pandas dataframe, pandas serie): output data for the validation
        
        X_test (pandas dataframe): data for the test
        
        y_test (pandas dataframe, pandas serie): output data for the test
        
        output_variable (str): name of the output variable
        
        model (e.g. SKlearn models): it is a trained model
        
        output_path (str): where to save the results
        
        data_training (pandas dataframe): entire training data
        
        data_test (pandas dataframe): entire test data
        
        grid_5km_shp (shape file): 5km grid saved as shape file provided to be able to save the results as shape files for the
        further illustrations as maps
        
        model (e.g. SKlearn models): it is a trained model (optional)
        
        
        Output:
        ---------------------------
        result (dictionary): a dictionary containing the performance of training, validation and test phase for 
        specified method. It contains MAE, MSE, R-squared
        
        y_pred (pandas dataframe): a dataframe containing the predictions
        
        model (e.g. SKlearn models): it is a trained model
    '''
    
    
    print(method + ' is running ... ')
    
    if not model:
        # If no trained model is provided, first, build and train the desired model
        model = Build_fit_model(method, X_train, y_train)
    
    # Validation and Evaluation of the model
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_val)
    y_pred = model.predict(X_test)
    y_pred = pd.DataFrame(y_pred, columns = [output_variable])

    result = {'Method': method, 'MSE_train': mean_squared_error(y_train, y_pred_train),
                            'MAE_train': mean_absolute_error(y_train, y_pred_train),
                            'R_2_train': r2_score(y_train, y_pred_train),
                            'MSE_val': mean_squared_error(y_val, y_pred_test),
                            'MAE_val': mean_absolute_error(y_val, y_pred_test),
                            'R_2_val': r2_score(y_val, y_pred_test), 'MSE_pred': mean_squared_error(y_test, y_pred),
                            'MAE_pred': mean_absolute_error(y_test, y_pred),
                            'R_2_pred': r2_score(y_test, y_pred)}
    print(method + ' results:')
    pprint(result)
    
#     Merge dataframes to make the final output dataset 
    data_result= pd.DataFrame()
    data_result = data_result.append(data_training)
    data_result = data_result.append(pd.concat([data_test.drop([output_variable], axis = 1),pd.DataFrame(y_pred)],axis=1))
    data_result = data_result[['id', output_variable]]
    data_result.to_csv((output_path + method + '/prediction_result_' + method + '.csv'), sep=',', index=False)
    
#     Merge dataframes to make the Ground Truth
    GT = pd.DataFrame()
    GT = GT.append(data_training)
    GT = GT.append(pd.concat([data_test.drop([output_variable], axis = 1),pd.DataFrame(y_test)],axis=1))
    GT = GT[['id', output_variable]]
    GT.to_csv((output_path + 'GT/GT.csv'), sep=',', index=False)   
    
#     Make the difference output
    data_diff_result= pd.DataFrame()
    data_diff_result = data_diff_result.append(data_training)
    data_diff_result[output_variable] = 0
    pred_variable = 'pred'+output_variable
    data_test = pd.concat([data_test, y_pred.rename(columns={output_variable: pred_variable})], axis=1)
    data_test[output_variable] = data_test[output_variable] - data_test[pred_variable]
    data_diff_result = data_diff_result.append(data_test.drop([pred_variable], axis = 1))
    data_diff_result = data_diff_result[['id', output_variable]]
    data_diff_result.to_csv((output_path + 'diff_' + method + '/prediction_result_diff_' + method + '.csv'), sep=',', index=False)
    
#     Make shp files
    merged_data = grid_5km_shp.merge(data_result, left_on="id", right_on="id")
#     merged_data_GT = grid_5km_shp.merge(GT, left_on="id", right_on="id")
    merged_data_diff = grid_5km_shp.merge(data_diff_result, left_on="id", right_on="id")
#     save the GeoDataFrame
    merged_data.to_file(driver = 'ESRI Shapefile', filename= output_path + method + '/' + method + '_shape.shp')
#     merged_data.to_file(driver = 'ESRI Shapefile', filename= output_path + 'GT/GT_shape.shp')
    merged_data_diff.to_file(driver = 'ESRI Shapefile', filename= output_path + 'diff_'+ method + '/diff_' + method + '_shape.shp')
    
    return result, y_pred, model

In [8]:
def Prediction_without_validation(method, X_train, y_train, X_test, y_test, output_variable, output_path, data_training,
               data_test, grid_5km_shp, model = None):
        
    ''' This function makes prediction for the provided data and reports three metrics, i.e. MAE, MSE, R-squared.
        
        Input:
        --------------------------
        
        method (str): name of the method
        
        X_train (pandas dataframe): data for the training
        
        X_val (pandas dataframe): data for the validation
        
        y_train (pandas dataframe, pandas serie): output data for the training
        
        y_val (pandas dataframe, pandas serie): output data for the validation
        
        X_test (pandas dataframe): data for the test
        
        y_test (pandas dataframe, pandas serie): output data for the test
        
        output_variable (str): name of the output variable
        
        model (e.g. SKlearn models): it is a trained model
        
        output_path (str): where to save the results
        
        data_training (pandas dataframe): entire training data
        
        data_test (pandas dataframe): entire test data
        
        grid_5km_shp (shape file): 5km grid saved as shape file provided to be able to save the results as shape files for the
        further illustrations as maps
        
        model (e.g. SKlearn models): it is a trained model (optional)
        
        
        Output:
        ---------------------------
        result (dictionary): a dictionary containing the performance of training, validation and test phase for 
        specified method. It contains MAE, MSE, R-squared
        
        y_pred (pandas dataframe): a dataframe containing the predictions
        
        model (e.g. SKlearn models): it is a trained model
    '''
    
    
    print(method + ' is running ... ')
    
    if not model:
        # If no trained model is provided, first, build and train the desired model
        model = Build_fit_model(method, X_train, y_train)
    
    # Evaluation of the model
    y_pred_train = model.predict(X_train)
    y_pred = model.predict(X_test)
    y_pred = pd.DataFrame(y_pred, columns = [output_variable])

    result = {'Method': method, 'MSE_train': mean_squared_error(y_train, y_pred_train),
                            'MAE_train': mean_absolute_error(y_train, y_pred_train),
                            'R_2_train': r2_score(y_train, y_pred_train),
                            'MSE_pred': mean_squared_error(y_test, y_pred),
                            'MAE_pred': mean_absolute_error(y_test, y_pred),
                            'R_2_pred': r2_score(y_test, y_pred)}
    print(method + ' results:')
    pprint(result)
    
#     Merge dataframes to make the final output dataset 
    data_result= pd.DataFrame()
    data_result = data_result.append(data_training)
    data_result = data_result.append(pd.concat([data_test.drop([output_variable], axis = 1),pd.DataFrame(y_pred)],axis=1))
    data_result = data_result[['id', output_variable]]
    data_result.to_csv((output_path + method + '/prediction_result_' + method + '.csv'), sep=',', index=False)
    
#     Merge dataframes to make the Ground Truth
    GT = pd.DataFrame()
    GT = GT.append(data_training)
    GT = GT.append(pd.concat([data_test.drop([output_variable], axis = 1),pd.DataFrame(y_test)],axis=1))
    GT = GT[['id', output_variable]]
    GT.to_csv((output_path + 'GT/GT.csv'), sep=',', index=False)   
    
#     Make the difference output
    data_diff_result= pd.DataFrame()
    data_diff_result = data_diff_result.append(data_training)
    data_diff_result[output_variable] = 0
    pred_variable = 'pred'+output_variable
    data_test = pd.concat([data_test, y_pred.rename(columns={output_variable: pred_variable})], axis=1)
    data_test[output_variable] = data_test[output_variable] - data_test[pred_variable]
    data_diff_result = data_diff_result.append(data_test.drop([pred_variable], axis = 1))
    data_diff_result = data_diff_result[['id', output_variable]]
    data_diff_result.to_csv((output_path + 'diff_' + method + '/prediction_result_diff_' + method + '.csv'), sep=',', index=False)
    
#     Make shp files
    merged_data = grid_5km_shp.merge(data_result, left_on="id", right_on="id")
#     merged_data_GT = grid_5km_shp.merge(GT, left_on="id", right_on="id")
    merged_data_diff = grid_5km_shp.merge(data_diff_result, left_on="id", right_on="id")
#     save the GeoDataFrame
    merged_data.to_file(driver = 'ESRI Shapefile', filename= output_path + method + '/' + method + '_shape.shp')
#     merged_data.to_file(driver = 'ESRI Shapefile', filename= output_path + 'GT/GT_shape.shp')
    merged_data_diff.to_file(driver = 'ESRI Shapefile', filename= output_path + 'diff_'+ method + '/diff_' + method + '_shape.shp')
    
    return result, y_pred, model

In [9]:
def Build_fit_all_methods(methods, X_train, y_train):
    
    ''' This function builds and fits bunch of models the provided data.
        
        Input:
        --------------------------
        methods (str): name of the method
        
        X_train (pandas dataframe): data for the predictors
        
        y_train (pandas dataframe, pandas serie): output data
        
        
        Output:
        ---------------------------
        models (list of e.g. sklearn model): list of trained model w.r.t. the specified methods
        
    '''
    
    models = []
    for method in methods:
        # Build and fit the model with the proper data
        models.append(Build_fit_model(method, X_train, y_train))
    
    return models